In [ ]:
import pandas as pd
import os

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

import random
import pandas as pd
import numpy as np
from scipy import stats
import math
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

if os.name == 'nt' :
    font_family = "Malgun Gothic"
else:
    font_family = "AppleGothic"
    
sns.set(font=font_family, rc ={"axes.unicode_minus" : False})

## 데이터 확인

In [ ]:
df = pd.read_csv('C:/Users/user/Desktop/생활인구 분석/생활이동/data/21년 10월/생활이동_행정동_2021.10_00시.csv', encoding='cp949')
df.head()

## 데이터 타입 확인

In [ ]:
df.dtypes

## 행정동 코드

In [ ]:
code = [
    1106071, #회기동
    1106072, # 휘경1동
    1106073, # 휘경2동
    1106080, # 청량리동
    1106081, # 용신동
    1106082, # 제기동
    1106083, # 전농1동
    1106084, # 전농2동
    1106086, # 답십리2동
    1106087, # 장안1동
    1106088, # 장안2동
    1106089, # 이문1동
    1106090, # 이문2동
    1106091  # 답십리1동
]

## 월별로 동대문구만 추출 후 통합
### 데이터가 많아서 한번에 통합이 어려워 나눠서 통합 진행

In [ ]:
def integrated(forder_name) :
    forders= os.listdir('C:/Users/user/Desktop/생활인구 분석/생활이동/data/'+forder_name)
    os.chdir('C:/Users/user/Desktop/생활인구 분석/생활이동/data/'+forder_name) 
    # 새로운 데이터 프레임 생성
    df_all = pd.DataFrame()

    code = [
        1106071, #회기동
        1106072, # 휘경1동
        1106073, # 휘경2동
        1106080, # 청량리동
        1106081, # 용신동
        1106082, # 제기동
        1106083, # 전농1동
        1106084, # 전농2동
        1106086, # 답십리2동
        1106087, # 장안1동
        1106088, # 장안2동
        1106089, # 이문1동
        1106090, # 이문2동
        1106091  # 답십리1동
    ]

    for files in forders:
        df= pd.read_csv(files, encoding='cp949')
        df = df[df['출발 행정동 코드'].isin(code) | df['도착 행정동 코드'].isin(code)]
        df_all= pd.concat([df_all, df])
    df_all.to_csv('C:/Users/user/Desktop/생활인구 분석/생활이동/data/'+forder_name+'/'+forder_name+' 통합.csv', encoding='cp949')

In [ ]:
for i in range(1, 10) :
    integrated('22년 '+ str(i) +'월')

In [ ]:
df_all.shape

## 하나로 통합
### 파일리스트 저장

In [ ]:
forders= os.listdir('C:/Users/user/Desktop/생활인구 분석/생활이동/data/월별 통합본')

### 현재 작업위치(working directory) 이동

In [ ]:
cd C:/Users/user/Desktop/생활인구 분석/생활이동/data/월별 통합본

### 데이터 통합

In [ ]:
# 새로운 데이터 프레임 생성
df_all = pd.DataFrame()

for files in forders:
    df= pd.read_csv(files, encoding='cp949')
    df_all= pd.concat([df_all, df])

In [ ]:
df_all = df_all.drop('Unnamed: 0', axis=1)

In [ ]:
df_all.to_csv('C:/Users/user/Desktop/생활인구 분석/생활이동/data/통합최종본.csv', encoding='cp949')

## 통합본 불러오기

In [ ]:
df_all = pd.read_csv('C:/Users/user/Desktop/생활인구 분석/생활이동/data/통합최종본.csv', encoding='cp949')

In [ ]:
df_all.head()
df_all = df_all.drop('Unnamed: 0', axis=1)

## null 확인

In [ ]:
df_all.isnull().sum()

## 동대문구 출발, 동대문구 도착 데이터프레임 나누기

In [ ]:
df_all_arrival = df_all[df_all['도착 행정동 코드'].isin(code)] # 동대문구 도착

In [ ]:
df_all_depart = df_all[df_all['출발 행정동 코드'].isin(code)] # 동대문구 출발

## 동대문구 출발 데이터프레임 분석

In [ ]:
df_all_depart.groupby('대상연월').size()

In [ ]:
label = ['10월', '11월', '12월', '1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월']

index = np.arange(len(label))

plt.figure(figsize=(15, 10))

plt.bar(index, df_all_depart.groupby('대상연월').size(), color='#e35f62')

plt.title('동대문구유출(월별이동량)', fontsize=20)

plt.xlabel('월', fontsize=15)

plt.ylabel('이동(백만)', fontsize=15)

plt.xticks(index, label, fontsize=13, rotation=0)

plt.show()

## 동대문구 도착 데이터프레임 분석

In [ ]:
df_all_arrival.groupby('대상연월').size()

In [ ]:
label = ['10월', '11월', '12월', '1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월']

index = np.arange(len(label))

plt.figure(figsize=(15, 10))

plt.bar(index, df_all_arrival.groupby('대상연월').size(), color='#e35f62')

plt.title('동대문구유입(월별이동량)', fontsize=20)

plt.xlabel('월', fontsize=15)

plt.ylabel('이동(백만)', fontsize=15)

plt.xticks(index, label, fontsize=13, rotation=0)

plt.show()

## 동대문 -> 다른 지역 상위 10개

In [ ]:
df_all_depart[df_all_depart['도착 행정동 코드'] == 1106083].count()

In [ ]:
df_all_depart_arrival = df_all_depart.groupby('도착 행정동 코드').size()
df_all_depart_arrival = df_all_depart_arrival.sort_values()
df_all_depart_arrival[-10:]

## 동대문구 -> 동대문구 외 지역 (유출)

In [ ]:
df_all_depart_out = df_all_depart[~df_all_depart['도착 행정동 코드'].isin(code)]

In [ ]:
df_all_depart_arrival = df_all_depart_out.groupby('도착 행정동 코드').size()
df_all_depart_arrival = df_all_depart_arrival.sort_values()
df_all_depart_arrival[-10:]

In [ ]:
label = ['서울특별시 중랑구 중화2동', '강원도', '서울특별시 성북구 종암동', '서울특별시 성동구 마장동', '서울특별시 성북구 석관동', 
         '서울특별시 중랑구 상봉2동', '서울특별시 성동구 왕십리도선동', '서울특별시 성북구 안암동', 
         '서울특별시 종로구 종로1·2·3·4가동', '서울특별시 성동구 용답동']

index = np.arange(len(label))

plt.figure(figsize=(15, 10))

plt.barh(index, df_all_depart_arrival[-10:], color='#e35f62')

plt.title('동대문구 유출 상위 10개 지역', fontsize=18)

plt.ylabel('행정동', fontsize=15)

plt.xlabel('생활이동(명)', fontsize=15)

plt.yticks(index, label, fontsize=13, rotation=0)

plt.show()

## 동대문구 외 지역 -> 동대문구 (유입)

In [ ]:
df_all_arrival_in = df_all_arrival[~df_all_arrival['출발 행정동 코드'].isin(code)]
df_all_arrival_in

In [ ]:
df_all_depart_arrival = df_all_arrival_in.groupby('출발 행정동 코드').size()
df_all_depart_arrival = df_all_depart_arrival.sort_values()
df_all_depart_arrival[-10:]

In [ ]:
label = ['강원도', '서울특별시 성동구 사근동', '서울특별시 성북구 종암동','서울특별시 성북구 석관동', '서울특별시 성동구 마장동',
        '서울특별시 중랑구 상봉2동', '서울특별시 성동구 왕십리도선동', '서울특별시 성북구 안암동', '서울특별시 성동구 용답동',
        '서울특별시 종로구 종로1·2·3·4가동']

index = np.arange(len(label))

plt.figure(figsize=(15, 10))

plt.barh(index, df_all_depart_arrival[-10:], color='#e35f62')

plt.title('동대문구 유입 상위 10개 지역', fontsize=18)

plt.ylabel('행정동', fontsize=15)

plt.xlabel('생활이동(명)', fontsize=15)

plt.yticks(index, label, fontsize=13, rotation=0)

plt.show()

## 시간대별 동대문구 유입 인구

In [ ]:
df_all_arrival_time = df_all_arrival_in.groupby('도착시간').size()
df_all_arrival_time

In [ ]:
label = []
for i in range(len(df_all_arrival_time)) :
    label.append(i)

index = np.arange(len(label))

plt.figure(figsize=(15, 10))

plt.bar(index, df_all_arrival_time, color='#e35f62')

plt.title('시간대별 동대문구 유입', fontsize=18)

plt.xlabel('시간', fontsize=15)

plt.ylabel('유입인원', fontsize=15)

plt.xticks(index, label, fontsize=13, rotation=0)

plt.show()

## 시간대별 동대문구 유출 인구

In [ ]:
df_all_depart_time = df_all_depart_out.groupby('도착시간').size()
df_all_depart_time

In [ ]:
label = []
for i in range(len(df_all_depart_time)) :
    label.append(i)

index = np.arange(len(label))

plt.figure(figsize=(15, 10))

plt.bar(index, df_all_depart_time, color='#e35f62')

plt.title('시간대별 동대문구 유출', fontsize=18)

plt.xlabel('시간', fontsize=15)

plt.ylabel('유출인원', fontsize=15)

plt.xticks(index, label, fontsize=13, rotation=0)

plt.show()

## 요일별 동대문구 유입인구

In [ ]:
df_all_arrival_day = df_all_arrival_in.groupby('요일').size()
df_all_arrival_day

In [ ]:
label = ['월요일', '화요일', '수요일', '목요일', '금요일','토요일', '일요일']

val = [3920001, 4322439, 4076543, 4115047, 4326185, 4069961, 3476292]

index = np.arange(len(label))

plt.figure(figsize=(15, 10))

plt.bar(index, val, color='#e35f62')

plt.title('요일별 동대문구유입', fontsize=18)

plt.xlabel('요일', fontsize=15)

plt.ylabel('유입인원', fontsize=15)

plt.xticks(index, label, fontsize=13, rotation=0)

plt.show()

## 요일별 동대문구 유출인구

In [ ]:
df_all_depart_day = df_all_depart_out.groupby('요일').size()
df_all_depart_day

In [ ]:
label = ['월요일', '화요일', '수요일', '목요일', '금요일','토요일', '일요일']

val = [3866183, 4287130, 4050331, 4097985, 4346529, 4131392, 3399506]

index = np.arange(len(label))

plt.figure(figsize=(15, 10))

plt.bar(index, val, color='#e35f62')

plt.title('요일별 동대문구 유출', fontsize=18)

plt.xlabel('요일', fontsize=15)

plt.ylabel('유입인원', fontsize=15)

plt.xticks(index, label, fontsize=13, rotation=0)

plt.show()

## 동대문구 -> 동대문구 (지역내 이동)

In [ ]:
df_all_arrival2 = df_all_arrival[df_all_arrival['출발 행정동 코드'].isin(code)]
df_all_arrival2 = df_all_arrival2.groupby('도착 행정동 코드').size()
df_all_arrival2 = df_all_arrival2.sort_values()
df_all_arrival2

In [ ]:
label = ['이문2동', '휘경2동', '청량리동', '전농2동', '답십리2동', '회기동', '이문1동', '휘경1동', '장안1동', '답십리1동', '용신동',
        '제기동', '장안2동', '전농1동']

index = np.arange(len(label))

plt.figure(figsize=(15, 10))

plt.barh(index, df_all_arrival2, color='#e35f62')

plt.title('동대문구내 유입', fontsize=18)


plt.ylabel('행정동', fontsize=15)

plt.xlabel('인원', fontsize=15)

plt.yticks(index, label, fontsize=13, rotation=0)

plt.show()